# Whisper語音辨識

## 載入套件

In [1]:
import whisper

## 載入模型及語音辨識

In [2]:
model = whisper.load_model("turbo")
result = model.transcribe("./whisper/darkness.mp3")
print(result["text"])

C:\Users\mikec\anaconda3\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


 Oh, you think darkness is your ally. You merely adopted the dark. I was born in it. Molded by it. I didn't see the light until I was already a man. By then it was nothing to me but light.


In [16]:
result = model.transcribe("./whisper/chinese.wav")
print(result["text"])

巴菲特罕见的警告美国面临财政愚蠢的风险如果财政愚蠢的风险盛行美国货币价值将增发


In [17]:
result = model.transcribe("./whisper/chinese.wav", language='zh')
print(result["text"])

巴菲特罕见的警告美国面临财政愚蠢的风险如果财政愚蠢的风险盛行美国货币价值将增发


## 低階API

In [28]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("./whisper/chinese.wav")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Detected language: zh
巴菲特罕见的警告,美国面临财政愚蠢的风险如果财政愚蠢的风险盛行,美国货币价值将增发


## 雲端 OpenAI API

## 生成文字稿(Transcription)

In [23]:
import openai
from openai import OpenAI

client = OpenAI()
audio_file= open("./whisper/chinese.wav", "rb")
transcript = client.audio.transcriptions.create(model="whisper-1", 
                                    file=audio_file) #, language='zh')
print(transcript.text)

巴菲特罕見的警告,美國面臨財政愚蠢的風險,如果財政愚蠢的風險盛行,美國貨幣價值將蒸發。


## 翻譯(translation)

In [24]:
audio_file= open("./whisper/chinese.wav", "rb")
transcript = client.audio.translations.create(model="whisper-1", 
                                  file=audio_file) #, language='zh')
print(transcript.text)

Buffett's rare warning, the United States is facing a risk of financial stupidity. If the risk of financial stupidity prevails, the value of U.S. currency will evaporate.


## 文字轉換為語音

In [26]:
audio_file= "./whisper/speech.wav"
response = openai.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input=transcript.text
)
response.stream_to_file(audio_file)

C:\Users\mikec\AppData\Local\Temp\ipykernel_9784\964533423.py:7: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(audio_file)


## 播放語音檔

In [27]:
from IPython.display import Audio

Audio(audio_file)